# How To: Orthorectify EMIT Data

>**Warning: This notebook uses a lot of memory, if using 2i2c, you will need a 29.7 GB, 3.75 CPU instance.**

EMIT Data is provided in non-orthorectified format to reduce data size. The `location` group contains latitude and longitude values of each pixel as well as a geometric lookup table (GLT) that can be used to orthocorrect the imagery. The GLT is an array that provides relative downtrack and crosstrack reference locations from the raw scene to facilitate fast projection of the dataset.

This notebook will walk through using the `emit_tools` module to do the orthorectification process as well as a in depth walkthrough of the process.

If you are planning to convert the EMIT `netCDF4` files to `.envi` format, the `reformat.py` available in the [emit-sds/emit-utils](https://github.com/emit-sds/emit-utils) repository can be used to orthorectify the imagery during the conversion process.

**Requirements:**
+ A NASA [Earthdata Login](https://urs.earthdata.nasa.gov/) account is required to download EMIT data   
+ Selected the `emit_tutorials` environment as the kernel for this notebook.
  + For instructions on setting up the environment, follow the the `setup_instructions.md` included in the `/setup/` folder of the repository.  

**Learning Objectives**
+ How to open an EMIT file as an `xarray.Dataset`
+ How to orthorectify an EMIT file

## 1. Using emit_xarray to Orthorectify

Import the required Python libraries.

In [1]:
# Import Packages
import os
import earthaccess
import netCDF4 as nc
from osgeo import gdal
import numpy as np
import xarray as xr
import hvplot.xarray
import holoviews as hv
import sys
sys.path.append('../modules/')
from emit_tools import emit_xarray

Login to your NASA Earthdata account and create a `.netrc` file using the `login` function from the `earthaccess` library. If you do not have an Earthdata Account, you can create one [here](https://urs.earthdata.nasa.gov/home). 

In [2]:
earthaccess.login(persist=True)

For this notebook we will download the files necessary using `earthaccess`. You can also access the data in place or stream it, but this can slow due to the file sizes. Provide a URL for an EMIT L2A Reflectance granule.

In [2]:
url = 'https://data.lpdaac.earthdatacloud.nasa.gov/lp-prod-protected/EMITL2ARFL.001/EMIT_L2A_RFL_001_20220903T163129_2224611_012/EMIT_L2A_RFL_001_20220903T163129_2224611_012.nc'

Get an HTTPS Session using your earthdata login, set a local path to save the file, and download the granule asset - This may take a while, the reflectance file is approximately 1.8 GB. 

In [3]:
# Get requests https Session using Earthdata Login Info
fs = earthaccess.get_requests_https_session()
# Retrieve granule asset ID from URL (to maintain existing naming convention)
granule_asset_id = url.split('/')[-1]
# Define Local Filepath
fp = f'../../data/{granule_asset_id}'
# Download the Granule Asset if it doesn't exist
if not os.path.isfile(fp):
    with fs.get(url,stream=True) as src:
        with open(fp,'wb') as dst:
            for chunk in src.iter_content(chunk_size=64*1024*1024):
                dst.write(chunk)

Use the `emit_xarray` function imported from the `emit_tools` module to read in the data.

The `emit_xarray` function will open and and place data from the groups of an EMIT dataset into an `xarray.Dataset`. It includes an `ortho` option which is set to `False` by default. This function reads the root group, `location` group, and `sensor_band_parameters` groups from the EMIT dataset. It then flattens all of the variables contained within those 3 groups into a single `xarray_dataset`. When `ortho=True`, it uses the GLT layers to build a lat/lon grid and places the values from the `reflectance` root group into the grid. The lat and lon grid is used as dimensional coordinates in the output `xarray.dataset` while the `wavelengths` and `fwhm` from the `sensor_band_parameters` group are used as non-dimensional coordinates. The `reflectance` data from the root group of the netCDF is then orthorectified and added as a variable in the `xarray.Dataset`.

Read in a dataset using the `emit_xarray` function.

In [4]:
ds = emit_xarray(fp, ortho=True)
ds

<xarray.Dataset> Size: 5GB
Dimensions:           (latitude: 2009, longitude: 2353, wavelengths: 285)
Coordinates:
  * wavelengths       (wavelengths) float32 1kB 381.0 388.4 ... 2.493e+03
    fwhm              (wavelengths) float32 1kB ...
    good_wavelengths  (wavelengths) float32 1kB ...
  * latitude          (latitude) float64 16kB -39.31 -39.31 ... -40.4 -40.4
  * longitude         (longitude) float64 19kB -62.51 -62.51 ... -61.24 -61.24
    elev              (latitude, longitude) float32 19MB -9.999e+03 ... -9.99...
    spatial_ref       int64 8B 0
Data variables:
    reflectance       (latitude, longitude, wavelengths) float32 5GB -9.999e+...
Attributes: (12/40)
    ncei_template_version:             NCEI_NetCDF_Swath_Template_v2.0
    summary:                           The Earth Surface Mineral Dust Source ...
    keywords:                          Imaging Spectroscopy, minerals, EMIT, ...
    Conventions:                       CF-1.63
    sensor:                            EMIT (Earth Surface Mineral Dust Sourc...
    instrument:                        EMIT
    ...                                ...
    spatial_ref:                       GEOGCS["WGS 84",DATUM["WGS_1984",SPHER...
    geotransform:                      [-6.25120945e+01  5.42232520e-04 -0.00...
    day_night_flag:                    Day
    title:                             EMIT L2A Estimated Surface Reflectance...
    granule_id:                        EMIT_L2A_RFL_001_20220903T163129_22246...
    Orthorectified:                    True

Let's visualize a red band. First assign fill-values to `np.nan`, then select and plot a band using the `sel` function from `xarray` and the `hvplot` library. When we orthorectify (place data on a grid and rotate it) we end up with a bunch of extra pixels that have no data. These are filled with -9999, which will distort our visualization unless we assign them to `np.nan` or mask them.

In [5]:
# Assign fill-values to nan
ds.reflectance.data[ds.reflectance.data == -9999] = np.nan
# Get red band and plot example
ds.sel(wavelengths=650,method='nearest').hvplot.image(cmap='viridis', aspect = 'equal', frame_width=500, rasterize=True)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


BokehModel(combine_events=True, render_bundle={'docs_json': {'ff572ec0-aaf3-4b39-bac7-6f3ad1ac1e6d': {'version…

The orthorectified dataset can also be saved as a netCDF4 output that can be reopened using the `xarray.open_dataset` function if so desired using the cell below.

In [6]:
ds.to_netcdf('../../data/example_emit_ortho_out.nc')
# Example for Opening
# ds = xr.open_dataset('../data/example_emit_ortho_out.nc')

Now open a dataset with `ortho=False`. Notice that the data is not on a Lat/Lon grid, instead each downtrack and crosstrack pixel center is geotagged with a latitude and longitude.

In [8]:
ds = emit_xarray(fp,ortho=False)
ds

<xarray.Dataset> Size: 2GB
Dimensions:           (downtrack: 1280, crosstrack: 1242, wavelengths: 285,
                       ortho_y: 2009, ortho_x: 2353)
Coordinates:
  * downtrack         (downtrack) int64 10kB 0 1 2 3 4 ... 1276 1277 1278 1279
  * crosstrack        (crosstrack) int64 10kB 0 1 2 3 4 ... 1238 1239 1240 1241
    lon               (downtrack, crosstrack) float64 13MB ...
    lat               (downtrack, crosstrack) float64 13MB ...
    elev              (downtrack, crosstrack) float64 13MB ...
    glt_x             (ortho_y, ortho_x) float64 38MB ...
    glt_y             (ortho_y, ortho_x) float64 38MB ...
  * wavelengths       (wavelengths) float32 1kB 381.0 388.4 ... 2.493e+03
    fwhm              (wavelengths) float32 1kB ...
    good_wavelengths  (wavelengths) float32 1kB ...
Dimensions without coordinates: ortho_y, ortho_x
Data variables:
    reflectance       (downtrack, crosstrack, wavelengths) float32 2GB ...
Attributes: (12/39)
    ncei_template_version:             NCEI_NetCDF_Swath_Template_v2.0
    summary:                           The Earth Surface Mineral Dust Source ...
    keywords:                          Imaging Spectroscopy, minerals, EMIT, ...
    Conventions:                       CF-1.63
    sensor:                            EMIT (Earth Surface Mineral Dust Sourc...
    instrument:                        EMIT
    ...                                ...
    spatialResolution:                 0.000542232520256367
    spatial_ref:                       GEOGCS["WGS 84",DATUM["WGS_1984",SPHER...
    geotransform:                      [-6.25120945e+01  5.42232520e-04 -0.00...
    day_night_flag:                    Day
    title:                             EMIT L2A Estimated Surface Reflectance...
    granule_id:                        EMIT_L2A_RFL_001_20220903T163129_22246...

We can see the difference in structure and the plotted data if the dataset is orthorectified or not. It's also not necessary to mask fill-values generated by the orthorectification process.

In [9]:
# Get red band and plot example
ds.sel(wavelengths=850,method='nearest').hvplot.image(cmap='viridis', aspect = 'equal', frame_width=500, rasterize=True)

BokehModel(combine_events=True, render_bundle={'docs_json': {'ce55e861-c4c7-463d-bc6f-781fcb6544fa': {'version…

## 2. Detailed Explanation of Orthorectification and xarray.Dataset Construction

As mentioned, the orthorectification process for EMIT data utilizes a premade geometric lookup table (GLT) that is included in the `location` group of the file. `glt_x` and `glt_y` contain the pixel indices from the raw dataset to construct an orthocorrected array. These indices data can be pulled from the raw dataset to populate an orthocorrected array using numpy broadcasting.

To do this, first read in the `location` group data. 

In [32]:
loc = xr.open_dataset(fp, engine = 'h5netcdf', group='location')

Next, build a numpy array using the GLT arrays from the `location` group. Use the `stack` function to stack the x and y GLT arrays and use the function `nan_to_num` to set the NaN values to the `GLT_NODATA_VALUE`, which is 0 for EMIT datasets. After doing this, you can check the shape of the array to see the expected height and width in pixels of the image after orthorectification.

In [33]:
GLT_NODATA_VALUE=0
glt_array = np.nan_to_num(np.stack([loc['glt_x'].data,loc['glt_y'].data],axis=-1),nan=GLT_NODATA_VALUE).astype(int)
glt_array.shape

(2009, 2353, 2)

After we have a GLT array, we need to open and create a numpy array from the reflectance dataset. Open the dataset using `xarray` and assign the data to an array. Here you can check the shape of the array to see the dimensions of the uncorrected data.

In [ ]:
ds = xr.open_dataset(fp,engine = 'h5netcdf')
ds_array = ds['reflectance'].data
ds_array.shape

(1280, 1242, 285)

: 

Now that we have both arrays, we need to assign a fill value for positions that have no data. We can then construct an empty array with the dimensions we desire (GLT shape) and populate it with values from the dataset. To do this we first use np.zeros to create an array of all zeros then add the `fill_value` to the array of zeros to change the zeros to the `fill_value`.

Next we can build an array of valid locations by omitting the portions of the array containing the `GLT_NODATA_VALUES`.

After that, change the indexing of the GLT array which is one based to zero based by subtracting 1 from them.

Lastly, we can use broadcasting/indexing to pull through the values we want from the dataset into our new output array.

In [ ]:
# Build Output Dataset
fill_value = -9999
out_ds = np.zeros((glt_array.shape[0], glt_array.shape[1], ds_array.shape[-1]), dtype=np.float32) + fill_value
valid_glt = np.all(glt_array != GLT_NODATA_VALUE, axis=-1)
# Adjust for One based Index
glt_array[valid_glt] -= 1
# Use indexing/broadcasting to populate array cells with 0 values
out_ds[valid_glt, :] = ds_array[glt_array[valid_glt, 1], glt_array[valid_glt, 0], :]
out_ds.shape

After this step, we have an array of values that is orthorectified, but if we want to have a grid of lat/lon values we still need to calculate them using the geotransform. We can retrieve this information from the dataset attributes (`ds.attrs['geotransform']`).

In [16]:
GT = ds.geotransform
GT

array([-6.25120945e+01,  5.42232520e-04, -0.00000000e+00, -3.93067591e+01,
       -0.00000000e+00, -5.42232520e-04])

The geotransform consists of 2 formulas with 6 coefficients used to calculate the xy-grid to project the dataset. 

`x_geo = GT[0] + x * GT[1] + y * GT[2]`  

`y_geo = GT[3] + x * GT[4] + y * GT[5]`

`GT[0]`  - The x-coordinate of the upper-left corner of the upper-left pixel  
`GT[1]`  - W-E pixel width  
`GT[2]`  - Row rotation (zero for North up images)  
`GT[3]`  - The y-coordinate of the upper-left corner of the upper-left pixel  
`GT[4]`  - Column rotation (zero for North up images)  
`GT[5]`  - N-S pixel height (negative value for a north-up image)  

Create empty arrays for the x and y (lon and lat) based on the dimensions of the dataset.


In [17]:
# Create Array for Lat and Lon and fill
dim_x = loc.glt_x.shape[1]
dim_y = loc.glt_x.shape[0]
lon = np.zeros(dim_x)
lat = np.zeros(dim_y)

After we have an empty array with the correct dimensions, we can populate it with values using the geotransform formula to build a lat/lon grid for plotting the orthorectified data. We can remove the `GT[2]` and `GT[4]` terms from the equation since our orthorectified image is North up. We also want to shift the geotransform by half a pixel since we want to build an array of pixel centers. We do this by adding half a pixel width to `GT[0]` and `GT[3]`.  Calclutate the latitude and longitude for each row (x_dim) and column (y_dim) of the dataset.

In [18]:
for x in np.arange(dim_x):
    x_geo = (GT[0]+0.5*GT[1]) + x * GT[1]
    lon[x] = x_geo
for y in np.arange(dim_y):
    y_geo = (GT[3]+0.5*GT[5]) + y * GT[5]
    lat[y] = y_geo

lat, lon

(array([-39.30703026, -39.3075725 , -39.30811473, ..., -40.3947487 ,
        -40.39529093, -40.39583316]),
 array([-62.51182342, -62.51128118, -62.51073895, ..., -61.23757699,
        -61.23703476, -61.23649253]))

As a last step we can place this and remaining information from the EMIT dataset, such as attributes and wavelengths into an `xarray.Dataset` if desired. To do this, build dictionaries for the coordinates and variables components of the dataset. In this example, the variable would be `reflectance`. First, we'll want to read in the `sensor_band_parameters` group to get the wavelength and fwhm info, then build the dictionaries.


In [19]:
wvl = xr.open_dataset(fp, engine = 'h5netcdf', group='sensor_band_parameters')
coords = {'lat':(['lat'],lat), 'lon':(['lon'],lon), **wvl.variables} ## ** upacks the existing dictionary from the wvl dataset.
data_vars = {'reflectance':(['lat','lon','bands'], out_ds)}

Now build an `xarray.Dataset` using the dictionaries, then add attributes to the dataset from the non-orthocorrected data. We also can add the crs using the `rio.write_crs` function, to allow `rasterio` and `rioxarray` to recognize the CRS for future use.

>Note: the wavelength and fwhm are included as non-dimensional coordinates because they are independent of Lat/Lon.

In [20]:
out_xr = xr.Dataset(data_vars=data_vars, coords=coords, attrs= ds.attrs)
out_xr['reflectance'].attrs = ds['reflectance'].attrs
out_xr.coords['lat'].attrs = loc['lat'].attrs
out_xr.coords['lon'].attrs = loc['lon'].attrs
out_xr.rio.write_crs(ds.spatial_ref,inplace=True) # Add CRS in easily recognizable format
out_xr

<xarray.Dataset> Size: 5GB
Dimensions:           (lat: 2009, lon: 2353, bands: 285)
Coordinates:
  * lat               (lat) float64 16kB -39.31 -39.31 -39.31 ... -40.4 -40.4
  * lon               (lon) float64 19kB -62.51 -62.51 -62.51 ... -61.24 -61.24
    wavelengths       (bands) float32 1kB ...
    fwhm              (bands) float32 1kB ...
    good_wavelengths  (bands) float32 1kB ...
    spatial_ref       int64 8B 0
Dimensions without coordinates: bands
Data variables:
    reflectance       (lat, lon, bands) float32 5GB -9.999e+03 ... -9.999e+03
Attributes: (12/38)
    ncei_template_version:             NCEI_NetCDF_Swath_Template_v2.0
    summary:                           The Earth Surface Mineral Dust Source ...
    keywords:                          Imaging Spectroscopy, minerals, EMIT, ...
    Conventions:                       CF-1.63
    sensor:                            EMIT (Earth Surface Mineral Dust Sourc...
    instrument:                        EMIT
    ...                                ...
    southernmost_latitude:             -40.39610428069674
    spatialResolution:                 0.000542232520256367
    spatial_ref:                       GEOGCS["WGS 84",DATUM["WGS_1984",SPHER...
    geotransform:                      [-6.25120945e+01  5.42232520e-04 -0.00...
    day_night_flag:                    Day
    title:                             EMIT L2A Estimated Surface Reflectance...

Now we have an orthorectified dataset with coordinates that can be plotted. We can also mask the `fill_values` of -9999 to make a clearer figure.

In [21]:
out_xr['reflectance'].data[out_xr['reflectance'].data == fill_value] = np.nan
out_xr.sel(bands=63).hvplot.image(cmap='viridis', aspect = 'equal', frame_width=500, rasterize=True)

BokehModel(combine_events=True, render_bundle={'docs_json': {'9a97ae80-41ff-43f9-a2ce-7aac72065899': {'version…

## Contact Info:  

Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: <https://lpdaac.usgs.gov/>  
Date last modified: 05-30-2023  

¹Work performed under USGS contract G15PD00467 for NASA contract NNG14HH33I.  